In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [3]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_eab848c2c8384710bfd4d52a3a8c56ce = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='5QJ7L6IcHehgbMzb1UN7p0RhO6L2SgjWneecP-Zwlqho',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_eab848c2c8384710bfd4d52a3a8c56ce.get_object(Bucket='ameyproject-donotdelete-pr-iyhtsqozclqsce',Key='Life_Expectancy.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()

,Year,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,2015,65.0,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,17.2,17.3,0.479,10.1
1,2014,59.9,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,17.5,17.5,0.476,10.0
2,2013,59.9,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,17.7,17.7,0.470,9.9
3,2012,59.5,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,17.9,18.0,0.463,9.8
4,2011,59.2,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,18.2,18.2,0.454,9.5


In [4]:
df.isnull().sum()

Year                                 0
Life expectancy                     10
Adult Mortality                     10
infant deaths                        0
Alcohol                            194
percentage expenditure               0
Hepatitis B                        553
Measles                              0
 BMI                                34
under-five deaths                    0
Polio                               19
Total expenditure                  226
Diphtheria                          19
 HIV/AIDS                            0
 thinness  1-19 years               34
 thinness 5-9 years                 34
Income composition of resources    167
Schooling                          163
dtype: int64

In [5]:
df.dtypes

Year                                 int64
Life expectancy                    float64
Adult Mortality                    float64
infant deaths                        int64
Alcohol                            float64
percentage expenditure             float64
Hepatitis B                        float64
Measles                              int64
 BMI                               float64
under-five deaths                    int64
Polio                              float64
Total expenditure                  float64
Diphtheria                         float64
 HIV/AIDS                          float64
 thinness  1-19 years              float64
 thinness 5-9 years                float64
Income composition of resources    float64
Schooling                          float64
dtype: object

In [6]:
col_names = [i for i in df.columns]
col_names

['Year',
 'Life expectancy ',
 'Adult Mortality',
 'infant deaths',
 'Alcohol',
 'percentage expenditure',
 'Hepatitis B',
 'Measles ',
 ' BMI ',
 'under-five deaths ',
 'Polio',
 'Total expenditure',
 'Diphtheria ',
 ' HIV/AIDS',
 ' thinness  1-19 years',
 ' thinness 5-9 years',
 'Income composition of resources',
 'Schooling']

In [8]:
df.dropna(inplace=True)

In [9]:
df.isnull().sum()

Year                               0
Life expectancy                    0
Adult Mortality                    0
infant deaths                      0
Alcohol                            0
percentage expenditure             0
Hepatitis B                        0
Measles                            0
 BMI                               0
under-five deaths                  0
Polio                              0
Total expenditure                  0
Diphtheria                         0
 HIV/AIDS                          0
 thinness  1-19 years              0
 thinness 5-9 years                0
Income composition of resources    0
Schooling                          0
dtype: int64

In [10]:
df.shape

(2083, 18)

In [11]:
df.rename(columns={" BMI ":"BMI","Life expectancy ":"Life_Expectancy","Adult Mortality":"Adult_Mortality",
                   "infant deaths":"Infant_Deaths","percentage expenditure":"Percentage_Exp","Hepatitis B":"HepatitisB",
                  "Measles ":"Measles"," BMI ":"BMI","under-five deaths ":"Under_Five_Deaths","Diphtheria ":"Diphtheria",
                  " HIV/AIDS":"HIV/AIDS"," thinness  1-19 years":"thinness_1to19_years"," thinness 5-9 years":"thinness_5to9_years","Income composition of resources":"Income_Comp_Of_Resources",
                   "Total expenditure":"Tot_Exp"},inplace=True)

In [12]:
col_dict = {'Life_Expectancy':1 , 'Adult_Mortality':2 ,
        'Alcohol':3 , 'Percentage_Exp': 4, 'HepatitisB': 5,
       'Measles' : 6, 'BMI': 7, 'Under_Five_Deaths' : 8, 'Polio' : 9, 'Tot_Exp' :10,
       'Diphtheria':11, 'HIV/AIDS':12,
       'thinness_1to19_years' :15, 'thinness_5to9_years' :16,
       'Income_Comp_Of_Resources' : 17, 'Schooling' :18, 'Infant_Deaths':19}

In [13]:
for variable in col_dict.keys():
    q75, q25 = np.percentile(df[variable], [75 ,25])
    iqr = q75 - q25
    min_val = q25 - (iqr*1.5)
    max_val = q75 + (iqr*1.5)
    print("Number of outliers in {} : {} ".format(variable,len((np.where((df[variable] > max_val) | (df[variable] < min_val))[0]))))

Number of outliers in Life_Expectancy : 59 
Number of outliers in Adult_Mortality : 75 
Number of outliers in Alcohol : 1 
Number of outliers in Percentage_Exp : 253 
Number of outliers in HepatitisB : 221 
Number of outliers in Measles : 378 
Number of outliers in BMI : 0 
Number of outliers in Under_Five_Deaths : 287 
Number of outliers in Polio : 178 
Number of outliers in Tot_Exp : 26 
Number of outliers in Diphtheria : 159 
Number of outliers in HIV/AIDS : 408 
Number of outliers in thinness_1to19_years : 87 
Number of outliers in thinness_5to9_years : 92 
Number of outliers in Income_Comp_Of_Resources : 66 
Number of outliers in Schooling : 46 
Number of outliers in Infant_Deaths : 254 


In [14]:
winsorize(df["Life_Expectancy"],(0.01,0), inplace=True)
winsorize(df["Adult_Mortality"],(0,0.03), inplace=True)
winsorize(df["Infant_Deaths"],(0,0.10), inplace=True)
winsorize(df["Alcohol"],(0,0.01), inplace=True)
winsorize(df["Percentage_Exp"],(0,0.12), inplace=True)
winsorize(df["HepatitisB"],(0.11,0), inplace=True)
winsorize(df["Measles"],(0,0.19), inplace=True)
winsorize(df["Under_Five_Deaths"],(0,0.12), inplace=True)
winsorize(df["Polio"],(0.09,0), inplace=True)
winsorize(df["Tot_Exp"],(0,0.01), inplace=True)
winsorize(df["Diphtheria"],(0.10,0), inplace=True)
winsorize(df["HIV/AIDS"],(0,0.16), inplace=True)
winsorize(df["thinness_1to19_years"],(0,0.04), inplace=True)
winsorize(df["thinness_5to9_years"],(0,0.04), inplace=True)
winsorize(df["Income_Comp_Of_Resources"],(0.05,0), inplace=True)
winsorize(df["Schooling"],(0.02,0.01), inplace=True)

masked_array(data=[10.1, 10. ,  9.9, ..., 10. ,  9.8,  9.8],
             mask=False,
       fill_value=1e+20)

In [15]:
for variable in col_dict.keys():
    q75, q25 = np.percentile(df[variable], [75 ,25])
    iqr = q75 - q25
    min_val = q25 - (iqr*1.5)
    max_val = q75 + (iqr*1.5)
    print("Number of outliers in {} : {} ".format(variable,len((np.where((df[variable] > max_val) | (df[variable] < min_val))[0]))))

Number of outliers in Life_Expectancy : 59 
Number of outliers in Adult_Mortality : 75 
Number of outliers in Alcohol : 0 
Number of outliers in Percentage_Exp : 253 
Number of outliers in HepatitisB : 0 
Number of outliers in Measles : 0 
Number of outliers in BMI : 0 
Number of outliers in Under_Five_Deaths : 287 
Number of outliers in Polio : 0 
Number of outliers in Tot_Exp : 26 
Number of outliers in Diphtheria : 0 
Number of outliers in HIV/AIDS : 408 
Number of outliers in thinness_1to19_years : 87 
Number of outliers in thinness_5to9_years : 92 
Number of outliers in Income_Comp_Of_Resources : 0 
Number of outliers in Schooling : 0 
Number of outliers in Infant_Deaths : 254 


In [16]:
df.head()

,Year,Life_Expectancy,Adult_Mortality,Infant_Deaths,Alcohol,Percentage_Exp,HepatitisB,Measles,BMI,Under_Five_Deaths,Polio,Tot_Exp,Diphtheria,HIV/AIDS,thinness_1to19_years,thinness_5to9_years,Income_Comp_Of_Resources,Schooling
0,2015,65.0,263.0,54,0.01,71.279624,65.0,605,19.1,64,63.0,8.16,66.0,0.1,15.3,15.4,0.479,10.1
1,2014,59.9,271.0,54,0.01,73.523582,62.0,492,18.6,64,63.0,8.18,66.0,0.1,15.3,15.4,0.476,10.0
2,2013,59.9,268.0,54,0.01,73.219243,64.0,430,18.1,64,63.0,8.13,66.0,0.1,15.3,15.4,0.470,9.9
3,2012,59.5,272.0,54,0.01,78.184215,67.0,605,17.6,64,67.0,8.52,67.0,0.1,15.3,15.4,0.463,9.8
4,2011,59.2,275.0,54,0.01,7.097109,68.0,605,17.2,64,68.0,7.87,68.0,0.1,15.3,15.4,0.454,9.5


In [17]:
X = df.drop(['Life_Expectancy'], axis=1).values
Y = df.iloc[:, 1].values.reshape(-1,1)

In [18]:
X.shape, Y.shape

((2083, 17), (2083, 1))

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state= 42)

In [20]:
model = LinearRegression(fit_intercept=True, normalize=True).fit(X_train, Y_train)

In [21]:
predictions= model.predict(X_test)

In [22]:
mean_squared_error(predictions, Y_test)

12.833904554965608

In [23]:
mean_absolute_error(predictions, Y_test)

2.5435169215233024

In [24]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2020-06-20 07:13:45,651 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.


In [25]:
wml_credentials = {    
  "apikey": "Qwg7NXlnJ6B2n-DEu4HQcSdiQr14THwEI0KjbJRKhJDN",
  "iam_apikey_description": "Auto-generated for key 45b803bb-9e5a-4c34-ba85-b8bc4b84af4d",
  "iam_apikey_name": "Service credentials-3",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/047d7e5db3284f559d41b37e6d5ea6ae::serviceid:ServiceId-bd772675-728c-4069-a2e2-98625bc71c9a",
  "instance_id": "516ce7de-7708-43d2-8e8a-c088cdf276e0",
  "url": "https://eu-gb.ml.cloud.ibm.com"
}

In [26]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [27]:
model_props = {
    client.repository.ModelMetaNames.AUTHOR_NAME : "Amey",
    client.repository.ModelMetaNames.AUTHOR_EMAIL : 'ameys2907@gmail.com',
    client.repository.ModelMetaNames.NAME : "Life_Expectancy"
}

In [28]:
model_artifact = client.repository.store_model(model, meta_props = model_props)

In [29]:
model_artifact

{'metadata': {'guid': 'badc1f5e-3eb9-4dac-b6e7-7fd75d4b4cb9',
  'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/516ce7de-7708-43d2-8e8a-c088cdf276e0/published_models/badc1f5e-3eb9-4dac-b6e7-7fd75d4b4cb9',
  'created_at': '2020-06-20T07:13:46.339Z',
  'modified_at': '2020-06-20T07:13:46.397Z'},
 'entity': {'runtime_environment': 'python-3.6',
  'learning_configuration_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/516ce7de-7708-43d2-8e8a-c088cdf276e0/published_models/badc1f5e-3eb9-4dac-b6e7-7fd75d4b4cb9/learning_configuration',
  'author': {'name': 'Amey'},
  'name': 'Life_Expectancy',
  'learning_iterations_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/516ce7de-7708-43d2-8e8a-c088cdf276e0/published_models/badc1f5e-3eb9-4dac-b6e7-7fd75d4b4cb9/learning_iterations',
  'feedback_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/516ce7de-7708-43d2-8e8a-c088cdf276e0/published_models/badc1f5e-3eb9-4dac-b6e7-7fd75d4b4cb9/feedback',
  'latest_version': {'url': 'htt

In [30]:
guid = client.repository.get_model_uid(model_artifact)

In [31]:
guid

'badc1f5e-3eb9-4dac-b6e7-7fd75d4b4cb9'

In [32]:
deploy = client.deployments.create(guid, name="Life_Exp")



#######################################################################################

Synchronous deployment creation for uid: 'badc1f5e-3eb9-4dac-b6e7-7fd75d4b4cb9' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='2fa3b583-22fe-4f47-bb7f-f827e6cd3afe'
------------------------------------------------------------------------------------------------




In [33]:
client.deployments.list()

------------------------------------  -----------------  ------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME               TYPE    STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
2fa3b583-22fe-4f47-bb7f-f827e6cd3afe  Life_Exp           online  DEPLOY_SUCCESS  2020-06-20T07:14:28.294Z  scikit-learn-0.20  model
f459bc13-04aa-48b9-9f42-a9cc3932835a  Salary_Prediction  online  DEPLOY_SUCCESS  2020-06-16T12:56:31.343Z  scikit-learn-0.20  model
------------------------------------  -----------------  ------  --------------  ------------------------  -----------------  -------------


In [34]:
deploy

{'metadata': {'guid': '2fa3b583-22fe-4f47-bb7f-f827e6cd3afe',
  'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/516ce7de-7708-43d2-8e8a-c088cdf276e0/deployments/2fa3b583-22fe-4f47-bb7f-f827e6cd3afe',
  'created_at': '2020-06-20T07:14:28.294Z',
  'modified_at': '2020-06-20T07:14:28.546Z'},
 'entity': {'runtime_environment': 'python-3.6',
  'name': 'Life_Exp',
  'scoring_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/516ce7de-7708-43d2-8e8a-c088cdf276e0/deployments/2fa3b583-22fe-4f47-bb7f-f827e6cd3afe/online',
  'deployable_asset': {'name': 'Life_Expectancy',
   'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/516ce7de-7708-43d2-8e8a-c088cdf276e0/published_models/badc1f5e-3eb9-4dac-b6e7-7fd75d4b4cb9',
   'guid': 'badc1f5e-3eb9-4dac-b6e7-7fd75d4b4cb9',
   'created_at': '2020-06-20T07:14:28.271Z',
   'type': 'model'},
  'description': 'Description of deployment',
  'status_details': {'status': 'DEPLOY_SUCCESS'},
  'model_type': 'scikit-learn-0.20',
  'status': 'DEPLO

In [35]:
scoring_url = client.deployments.get_scoring_url(deploy)

In [36]:
scoring_url

'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/516ce7de-7708-43d2-8e8a-c088cdf276e0/deployments/2fa3b583-22fe-4f47-bb7f-f827e6cd3afe/online'